In [1]:
import re
import nltk, unicodedata
import time
# import multiprocess as mp       # mac platform only
import multiprocessing as mp     # linux platform
import os
import string


in_text = "/fairseq/examples/wav2vec/manifest-u2/corpus/librispeech_lm_corpus.minus_librivox.metadata_and_manual_and_missing.corpus.txt"

with open(in_text, 'r') as rf:
    text = rf.readlines()

KeyboardInterrupt: 

In [ ]:
_rdecode = dict(zip('XVI', (10, 5, 1)))
def decode(roman):
    result = 0
    for r, r1 in zip(roman, roman[1:]):
        rd, rd1 = _rdecode[r], _rdecode[r1]
        result += -rd if rd < rd1 else rd
    return result + _rdecode[roman[-1]]

def convert_roman(text):
    """
    Uses heuristics to decide whether to convert a string that looks like a
    roman numeral to decimal number.
    """
    lines = re.split('\r?\n', text)
    new_lines = list()
    for i, l in enumerate(lines):
        m = re.match('^(\s*C((hapter)|(HAPTER))\s+)(([IVX]+)|([ivx]+))(.*)', l)
        if m is not None:
            new_line = "%s%s%s" % (m.group(1), decode(m.group(5).upper()), m.group(8))
            new_lines.append(new_line)
            continue
        m = re.match('^(\s*)(([IVX]+)|([ivx]+))([\s\.]+[A-Z].*)', l)
        if m is not None:
            new_line = "%s%s%s" % (m.group(1), decode(m.group(2).upper()), m.group(5))
            new_lines.append(new_line)
            continue
        new_lines.append(l)
    return '\n'.join(new_lines)

In [ ]:
digit_sent = re.compile("^[0-9].*[0-9]$")
begin_with = re.compile("^[\s(_\[]")
end_with = re.compile("_\.?$")
author = re.compile("^By ")


def regex(line):
    return re.match(begin_with, line) or re.match(end_with, line) \
        or re.match(digit_sent, line) or re.match(author, line) 

# http://rosettacode.org/wiki/Roman_numerals/Decode#Python

def paragraph(text):
    sentences = []
    temp = ""
    for line_num in range(len(text)):
        if text[line_num] != '\n' and not text[line_num].isupper():
            temp += text[line_num].rstrip() + " "
        else:
            temp = temp.rstrip() 
            if regex(temp):
                temp = ""
                continue
            if temp:
                temp += '\n'
                sentences.append(temp)
                temp = ""
    return sentences

In [ ]:
punkt = nltk.data.load('tokenizers/punkt/english.pickle')
punc = string.punctuation.replace("'","")

def segment_sentences(text):
    sents = punkt.tokenize(text)
    line_sents = [re.sub('\r?\n', ' ', s) for s in sents]
    line_sep = '  \n'
    return (line_sep.join(line_sents))

def remove_punctuation(line):
    line = line.replace('-', ' ').replace('/', ' ')
    return line.translate(str.maketrans('', '', punc)).strip()

def long2short(line):
    return [i.rstrip().upper() for i in line.split('\n')]

In [ ]:
sentences = paragraph(text)
sentences = [ unicodedata.normalize('NFKD', sentence)
                        for sentence in sentences ]

In [ ]:
stereo = sentences[:10]

with mp.Pool(mp.cpu_count()) as p:
    sentences = p.map(convert_roman, sentences)
    sentences = p.map(segment_sentences, sentences)
    # split a long sentence into short sentences
    sentences = p.map(remove_punctuation, sentences)
    sentences = p.map(long2short, sentences)

KeyboardInterrupt: 

In [ ]:
with open('/fairseq/examples/wav2vec/manifest-u2/corpus/normalized-librivox.txt', 'w') as wf:
    for para in sentences:
        for sent in para:
            if not sent.isdigit() and len(sent) > 3:
                print(sent, file=wf)